In [9]:
import os
from muselsl import stream, list_muses, view, record
from multiprocessing import Process
from time import strftime, gmtime

In [10]:
muses = list_muses()
stream_process = Process(target=stream, args=(muses[0]['address'],))
stream_process.start()
view()

BGAPIError: Unable to auto-detect BLED112 serial port

In [11]:
DURATION = 15  # secs
RECORDING_PATH = os.path.join("eeg-records", "recording-%s.csv" % strftime("%Y-%m-%d-%H.%M.%S", gmtime()))

In [7]:
recording_process = Process(target=record, args=(DURATION, RECORDING_PATH))
recording_process.start()

NameError: name 'DURATION' is not defined